In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [ ]:
def get_parent(term):
    url = f'https://amigo.geneontology.org/amigo/term/{term}#display-lineage-tab'
    response = requests.get(url).text
    soup = BeautifulSoup(response, 'html.parser')

    try: 
        parent_element = soup.find('span', style='font-weight: bold;').get_text().split('\xa0')
        return parent_element
    except:
        return None

In [ ]:
def get_children(term):
    url = f'https://amigo.geneontology.org/amigo/term/{term}#display-lineage-tab'
    response = requests.get(url).text
    soup = BeautifulSoup(response, 'html.parser')

    try:
        elements = soup.find('img', title=re.compile(r'Current term')).find_all_next('a')

        children = []
        for element in elements:
            if element.get_text().startswith('GO:'):
                
                children.append(element.get_text().split('\xa0'))
        return children

    except:
        return None

b = get_children('GO:0008152')

In [ ]:
import pandas as pd

df = pd.DataFrame()

all_go_ids = ['GO:0008152']
processed_ids = []

while all_go_ids:
    current_id = all_go_ids[0]
    all_go_ids.remove(current_id)
    
    if current_id in processed_ids:
        continue  # Skip if already processed

    child_data = get_children(current_id)
    if child_data is not None:
        child_df = pd.DataFrame(child_data, columns=['Child GO ID', 'Child Term'])

        parent_data = get_parent(current_id)
        if parent_data is not None:
            child_df['Parent GO ID'] = parent_data[0]
            child_df['Parent Term'] = parent_data[1]

        if not child_df.empty:
            df = pd.concat([df, child_df], ignore_index=True)

        # Add child IDs to the processing queue
        all_go_ids += child_df['Child GO ID'].tolist()
        print(all_go_ids)

    processed_ids.append(current_id)
    print("Current ID:", current_id)

# Now, df contains the desired DataFrame


In [ ]:
df.to_csv('go_hierarchy.csv', index=False)